# Week 8 - Clustering 

## Learning Objectives
+ Understanding High-dimensional data
+ t-SNE on data to understand and reduce dimensions of data
+ Hierarchical clustering
+ K-Means clustering
    + Difference with hierarchical clustering
    + Methods of finding number of clusters:
        + Using silhoutte analysis
        + Using elbow method

The contents of this tutorial are based on the chapter 10 of "An Introduction to Statistical Learning" by James G. et. al., python [notebook](https://nbviewer.jupyter.org/github/JWarmenhoven/ISL-python/blob/master/Notebooks/Chapter%2010.ipynb#Lab-3:-NCI60-Data-Example) on same chapter, sklearn [tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py) on k-means, sklearn [tutorial](https://scikit-learn.org/stable/modules/clustering.html) on clustering, sklearn [tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html) on agglomerative clustering, and introduction to dimensionality reduction [tutorial](https://www.datacamp.com/community/tutorials/introduction-t-sne).

## Loading the NCI-160 dataset

NCI60 is cancer cell line microarray data, which consists of 6,830 gene expression measurements on 64 cancer cell lines. So, our data is 64x6830, implying that the n<\<p. In case of such high-dimensional data, methods such as regression do not remain applicable. Dimensionality reduction is also useful for such data, before applying different ML models. 

Gene data is usually always a high-dimensionality data, and is useful in precision medicine - i.e. to recommend personalized treatment to patients. A common approach for such personalized treatment is categorizing the patients into various sub-types. For such sub-typing, clustering is usually performed and can be used to identify patients who are similar - to analyze their treatments and decide the further course of treatment for the patient under consideration.  

In our data, though we have cancer type label for each cell-line, in real-life data, we typically do not have this for all the patients, making the clustering quite important. Usually clustering methods are then analyzed on various datasets, and validation from the domain - such as the cancer types in our case are used to identify the best technique to be used. 

Our data has gene expression measurement which is usually achieved by quantifying levels of the gene product, which is often a protein.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rng = np.random.RandomState(10)

In [ ]:
df = pd.read_csv('nci.data.csv', skiprows=1, header=None).T

In [ ]:
df.columns = df.iloc[0]                                 # Set new column names
df.drop(0,inplace=True)                                 # Drop duplicated row

In [ ]:
labs = pd.read_csv('nci.label.txt', names=['type'], dtype="category")

In [ ]:
labs.type = labs.type.str.strip()

# Hierarchical Clustering

Let us first try to find out whether or not the observations cluster into distinct types of cancer. 

Here we have an option to standardize the variables to have mean zero and standard deviation one. However, this step is optional and should be performed only if we want each gene to be on the same scale. We could reasonably argue that it is better not to scale genes. However, it is a choice to make as a data analyst, and also evaluate experimentally.

## Agglomerative Clustering
This is a bottom-up clustering, and is built starting from the leaves and combining clusters up to the trunk. Each dendogram will represent each of our 64 samples. As we move up the tree, some leaves begin to fuse into branches. These correspond to
observations that are similar to each other. As we move higher up the tree, branches themselves fuse, either with leaves or other branches. The earlier (lower in the tree) fusions occur, the more similar the groups of observations are to each other. Thus, the height of the fusion depicts how different the observations are.

The [```AgglomerativeClustering```](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) class in sklearn implements four different types of linkages. The linkage criterion determines which distance to use between sets of observation. The algorithm will merge the pairs of cluster that minimize this criterion.
+ ‘ward’ minimizes the variance of the clusters being merged.
+ ‘average’ uses the average of the distances of each observation of the two sets.
+ ‘complete’ or ‘maximum’ linkage uses the maximum distances between all observations of the two sets.
+ ‘single’ uses the minimum of the distances between all observations of the two sets.

The default is ward. Usually complete linkage is used for Gene data.

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
%matplotlib inline

Let us write a function to plot dendogram. We use the [```cluster.hierarchy.dendogram```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram). The [```cluster.hierarchy```]() is available in scipy which has functions that cut hierarchical clusterings into flat clusterings or find the roots of the forest formed by a cut by providing the flat cluster ids of each observation. We will see usage of both these API - the sklearn and the scipy.

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

Now we can use the scipy API and also scale the gene data and see the clusters. 

In [ ]:
from scipy.cluster import hierarchy
from sklearn.preprocessing import scale

We see that the choice of linkage
certainly does affect the results obtained. Typically, single linkage will tend
to yield trailing clusters: very large clusters onto which individual observations attach one-by-one. On the other hand, complete and average linkage
tend to yield more balanced, attractive clusters. For this reason, complete
and average linkage are generally preferred to single linkage. 

In the complete linkage, we can set the threshold to get four clusters.

# K-Means for same number of clusters
We can perform K-Means with same number of clusters and check the clusters that are formed. 

In K Means clustering, since we start with random choice of clusters, the results produced by running the algorithm multiple times might differ. While results are reproducible in Hierarchical clustering.

In [ ]:
from sklearn.cluster import KMeans

We see that though clustering has been done in same number of clusters - the observations per cluster is different and the clusters themselves must also be different. You can do visualizations along with the cancer types to actually see the difference between the clusters using both algorithms.

# Finding k 
Unlike the hierarchical clustering, the k-means clustering has the problem of specifying the number of clusters (k). How do we find this? We have two popular methods for this approach:
+ Silhoutte Method
+ Elbow Method

## Silhoutte method
Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of \[-1, 1\].

Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.

### Performing dimensionality reduction - PCA
Sometimes performing clustering on the first few principal component score vectors can give better results than performing clustering on the full data. In this situation, we might view the principal component step as one of denoising the data. You can ofcourse perform the clustering without the PCA, however, on this specific data, the PCA helps in finding more meaningful clusters using K-means. So, let us first perform PCA on the data.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(random_state=rng)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df2_plot = pd.DataFrame(pca.fit_transform(df))

Now once we have this principal components, the 64 dimensional data can be used to perform silhoutte analysis. 

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

In [ ]:
def silhoutte_plot(n_clusters):
    fig, ax = plt.subplots()

    # The silhouette coefficient can range from -1, 1, but ranges from -0.1 to 1 for our data
    ax.set_xlim([-0.1, 1])
    
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax.set_ylim([0, len(df2_plot) + (n_clusters + 1) * 10])

    clusterer = KMeans(n_clusters=n_clusters, n_init=50, random_state=rng)
    cluster_labels = clusterer.fit_predict(df2_plot)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(df2_plot, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df2_plot, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
        
    ax.set_title("The silhouette plot for the various clusters.")
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")

    # Labeling the clusters
    centers = clusterer.cluster_centers_

    plt.title(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

## Elbow Method
This is another method used for selecting the number of clusters. We fit the model with range of K values. If the line chart resembles an arm, then the “elbow” (the point of inflection on the curve) is a good indication that the underlying model fits best at that point. 

### Performing dimensionality reduction - tSNE
t-Distributed Stochastic Neighbor Embedding (t-SNE) is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It is extensively applied in image processing, NLP, genomic data and speech processing. In simple terms, t-SNE minimizes the divergence between two distributions: a distribution that measures pairwise similarities of the input objects and a distribution that measures pairwise similarities of the corresponding low-dimensional points in the embedding. This is a particularly useful technique for visualizing high-dimensional data.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, random_state=rng)
df3 = pd.DataFrame(tsne.fit_transform(df), columns=['tsne1', 'tsne2'])

To find the optimal number of clusters - we plot the sum of squared distances of samples to their closest cluster center, available as ```inertia_``` attribute of the K-Means class. 

In [ ]:
sse = []
k_list = range(1, 20)
for k in k_list:
    km = KMeans(n_clusters=k, random_state=rng)
    km.fit(df3)
    sse.append(km.inertia_)
    
tsne_results_scale = pd.DataFrame({'Cluster': range(1,20), 'SSE': sse})

t-SNE often fails to preserve the global geometry of the data. This means that the relative position of clusters on the t-SNE plot is almost arbitrary and depends on random initialisation more than on anything else. While this may not be a problem in some situations, scRNA-seq data sets often exhibit biologically meaningful hierarchical structure, e.g. encompass several very different cell classes, each further divided into various types. Typical t-SNE plots do not capture such global structure, yielding a suboptimal and potentially misleading visualisation. So, t-SNE may not always yeild best dimensionality reduction for genetic data, however, it has shown very good results for high-dimensional data in general.

### How would we analyze which clustering is most representative for this type of data?
Usually the domain knowledge of the cancer types and which algorithm has given us good clusters which are most representative is evaluated. Another approach used for gene data is to evaluate the different patient characteristics, such as Age, co-morbidities, etc. to find if the clusters are actually identifying such patterns. Also, using multiple datasets is usually required in such unsupervised tasks. 

# Practice Exercise (Optional)
1. Can you plot the t-SNE and PCA with the cancer type labels and try to reason which dimensionality reduction might be more appropriate for our data? 
2. How does clustering vary when using hierarchical clustering on few principal components? Are these results different from the ones we obtain?
3. In this tutorial, we explored data where n<<p,what happens when n>>p? What might be the bottlenecks for doing k-means? The concept of batching might seem useful in this case. You can use [```make_blobs```](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html) to generate synthetic data for clustering and explore using the [```MiniBatchKMeans```](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html#sklearn.cluster.MiniBatchKMeans) algorithm. 